In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
path = r'C:\Users\Ben\Desktop\Career Foundry Tasks\Instacart Basket Analysis 02-2023'

In [3]:
ords_prods_cust = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_cust.pkl'))

In [4]:
# Security Implications

PII data exists because the data includes the names of people.

In [5]:
ords_prods_cust.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,Unnamed: 0,...,First Name,Last Name,Gender,State,Age,date_joined,n_dependants,marital_status,income,_merge
0,2539329,1,1,2,8,NaN,196,1,0,195,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
1,2398795,1,2,3,7,15.0,196,1,1,195,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
2,473747,1,3,3,12,21.0,196,1,1,195,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
3,2254736,1,4,4,7,29.0,196,1,1,195,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
4,431534,1,5,4,15,28.0,196,1,1,195,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both


In [17]:
ords_prods_cust['State'].value_counts().sort_index()

Alabama                 638003
Alaska                  648495
Arizona                 653964
Arkansas                636144
California              659783
Colorado                639280
Connecticut             623022
Delaware                637024
District of Columbia    613695
Florida                 629027
Georgia                 656389
Hawaii                  632901
Idaho                   607119
Illinois                633024
Indiana                 627282
Iowa                    625493
Kansas                  637538
Kentucky                632490
Louisiana               637482
Maine                   638583
Maryland                626579
Massachusetts           646358
Michigan                630928
Minnesota               647825
Mississippi             632675
Missouri                640732
Montana                 635265
Nebraska                625813
Nevada                  636139
New Hampshire           615378
New Jersey              627692
New Mexico              654494
New York

In [24]:
# Assign States to Regions

NE = ["Maine", "New Hampshire", "Vermont", "Massachusetts", "Rhode Island", "Connecticut", "New York", "Pennsylvania", "New Jersey"]
MW = ["Wisconsin", "Michigan", "Illinois", "Indiana", "Ohio", "North Dakota", "South Dakota", "Nebraska", "Kansas", "Minnesota", "Iowa", "Missouri"]
S = ["Delaware", "Maryland", "District of Columbia", "Virginia", "West Virginia", "North Carolina", "South Carolina", "Georgia", "Florida", "Kentucky", "Tennessee", "Mississippi", "Alabama", "Oklahoma", "Texas", "Arkansas", "Louisiana"]
W = ["Idaho", "Montana", "Wyoming", "Nevada", "Utah", "Colorado", "Arizona", "New Mexico", "Alaska", "Washington", "Oregon", "California", "Hawaii"]

In [25]:
# Create Region column with the regions assigned based off of the state

ords_prods_cust.loc[ords_prods_cust['State'].isin(NE), 'Region'] = 'Northeast'
ords_prods_cust.loc[ords_prods_cust['State'].isin(MW), 'Region'] = 'Midwest'
ords_prods_cust.loc[ords_prods_cust['State'].isin(S), 'Region'] = 'South'
ords_prods_cust.loc[ords_prods_cust['State'].isin(W), 'Region'] = 'West'

In [26]:
# Verify all data has a valid region

ords_prods_cust['Region'].value_counts(dropna = False)

South        10791885
West          8292913
Midwest       7597325
Northeast     5722736
Name: Region, dtype: int64

In [30]:
# Spending habits based off of region

crosstab = pd.crosstab(ords_prods_cust['Region'], ords_prods_cust['spender_flag'], dropna = False)

In [31]:
crosstab

spender_flag,High spender,Low spender
Region,,
Midwest,155975,7441350
Northeast,108225,5614511
South,209691,10582194
West,160354,8132559


In [32]:
ords_prods_cust.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32404859 entries, 0 to 32404858
Data columns (total 36 columns):
 #   Column                    Dtype   
---  ------                    -----   
 0   order_id                  int64   
 1   user_id                   int64   
 2   order_number              int64   
 3   order_dow                 int64   
 4   order_hour_of_day         int64   
 5   days_since_prior_order    float64 
 6   product_id                int64   
 7   add_to_cart_order         int64   
 8   reordered                 int64   
 9   Unnamed: 0                int64   
 10  product_name              object  
 11  aisle_id                  int64   
 12  department_id             int64   
 13  prices                    float64 
 14  merge                     category
 15  price_range_loc           object  
 16  busiest_day               object  
 17  busiest_days              object  
 18  busiest_hours_of_the_day  object  
 19  max_order                 int64   
 20  

In [34]:
ords_prods_cust.loc[ords_prods_cust['max_order'] < 5, 'exclusion_flag'] = 'exclude'

In [35]:
ords_prods_cust.loc[ords_prods_cust['max_order'] >= 5, 'exclusion_flag'] = 'include'

In [36]:
ords_prods_cust['exclusion_flag'].value_counts(dropna = False)

include    30964564
exclude     1440295
Name: exclusion_flag, dtype: int64

In [37]:
df_high_activity = ords_prods_cust.loc[ords_prods_cust['exclusion_flag'] == 'include']

In [38]:
df_high_activity.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30964564 entries, 0 to 32404854
Data columns (total 37 columns):
 #   Column                    Dtype   
---  ------                    -----   
 0   order_id                  int64   
 1   user_id                   int64   
 2   order_number              int64   
 3   order_dow                 int64   
 4   order_hour_of_day         int64   
 5   days_since_prior_order    float64 
 6   product_id                int64   
 7   add_to_cart_order         int64   
 8   reordered                 int64   
 9   Unnamed: 0                int64   
 10  product_name              object  
 11  aisle_id                  int64   
 12  department_id             int64   
 13  prices                    float64 
 14  merge                     category
 15  price_range_loc           object  
 16  busiest_day               object  
 17  busiest_days              object  
 18  busiest_hours_of_the_day  object  
 19  max_order                 int64   
 20  

In [39]:
df_low_activity = ords_prods_cust.loc[ords_prods_cust['exclusion_flag'] == 'exclude']

In [40]:
df_high_activity.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'high_activity.pkl'))

In [41]:
df_low_activity.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'low_activity.pkl'))

In [49]:
df_high_activity[["Age", "income", "n_dependants"]].describe()

,Age,income,n_dependants
count,3.096456e+07,3.096456e+07,3.096456e+07
mean,4.946803e+01,9.967587e+04,1.501819e+00
std,1.848528e+01,4.314187e+04,1.118896e+00
min,1.800000e+01,2.590300e+04,0.000000e+00
25%,3.300000e+01,6.729200e+04,1.000000e+00
50%,4.900000e+01,9.676500e+04,2.000000e+00
75%,6.500000e+01,1.281020e+05,3.000000e+00
max,8.100000e+01,5.939010e+05,3.000000e+00


In [52]:
ords_prods_cust['order_dow'].value_counts(dropna = False)

0    6204182
1    5660230
6    4496490
2    4213830
5    4205791
3    3840534
4    3783802
Name: order_dow, dtype: int64